In [1]:
import sys
sys.path.append('../')
import matplotlib.pyplot as plt
from smodels.decomposition.theorySMS import TheorySMS
from smodels.experiment.expSMS import ExpSMS
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.experiment.defaultFinalStates import finalStates
from smodels.base.model import Model
from collections import OrderedDict
from smodels.base.physicsUnits import fb, GeV
from itertools import product
from smodels.experiment.databaseObj import Database
import copy

### Get database

In [2]:
db = Database('official')

In [3]:
stop = False
fails = []
for exp in db.getExpResults():
    for tx in exp.getTxNames():
        try:
            tx.sortSMSMap()
        except:
            fails.append(exp.globalInfo.id+'-'+str(tx))
            # print('Failed for:')
            # print(exp.globalInfo.id,tx)
            # print(tx.constraint)
            # stop = True
            # break
    # if stop: break
print(len(set(fails)), 'fails')

ERROR in txnameObj.sortSMSMap() in 260: Can not set the SMS in T6WZh to the standard format.
Make sure the constraint [[[jet],[Z]],[[jet],[W]]]+[[[jet],[higgs]],[[jet],[W]]]+[[[jet],[W]],[[jet],[W]]]+[[[jet],[Z]],[[jet],[Z]]]+[[[jet],[higgs]],[[jet],[higgs]]]+[[[jet],[Z]],[[jet],[higgs]]] is consistent with the data grid for all the SMS appearing in the constraint.


Failed for:
ATLAS-SUSY-2016-07 T6WZh
[[[jet],[Z]],[[jet],[W]]]+[[[jet],[higgs]],[[jet],[W]]]+[[[jet],[W]],[[jet],[W]]]+[[[jet],[Z]],[[jet],[Z]]]+[[[jet],[higgs]],[[jet],[higgs]]]+[[[jet],[Z]],[[jet],[higgs]]]
1 fails


In [4]:
set(fails)

{'ATLAS-SUSY-2016-07-T6WZh'}